<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#현재-상태" data-toc-modified-id="현재-상태-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>현재 상태</a></span><ul class="toc-item"><li><span><a href="#Stacking" data-toc-modified-id="Stacking-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Stacking</a></span></li></ul></li></ul></div>

In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import pymysql
import warnings
warnings.filterwarnings(action='ignore')


def load_datasets(path) -> pd.DataFrame:
    train = pd.read_csv(os.path.join(path, '한국가스공사_시간별 공급량_20181231.csv'),
                        encoding='cp949')
    test = pd.read_csv(os.path.join(path, 'test.csv'))
    # test 데이터 전처리
    test['연월일'] = test['일자|시간|구분'].str.split(' ', expand=True)[0]
    test['시간'] = test['일자|시간|구분'].str.split(' ', expand=True)[1].astype(int)
    test['구분'] = test['일자|시간|구분'].str.split(' ', expand=True)[2]
    del test['일자|시간|구분']

    data = pd.concat([train, test], axis=0)
    data['연월일'] = pd.to_datetime(data['연월일'])
    return data

def load_weather() -> pd.DataFrame:
    db = pymysql.connect(host='localhost', port=3306, user='younghun', password='watson1259',
                        db='dacon_gas_weather_db', charset='utf8')
    cursor = db.cursor()
    sql = "SELECT datetime, avg_temp, min_temp, max_temp,\
              NULLIF(sum_rain, '') as sum_rain, avg_wind, avg_humid,\
              sum_gsr, NULLIF(ddmefs, '') as ddmefs, avg_ts \
              FROM weather ORDER BY datetime"
    
    weather = pd.read_sql(sql, db)
    weather = weather.fillna(0.)
    
    return weather
    
def merge_gas_weather(path):
    gas = load_datasets(path)
    weather = load_weather()
    gas_weather = gas.merge(weather, how='left', left_on='연월일', right_on='datetime')
    del gas_weather['datetime']
    
    return gas_weather

def make_datetime_vars(data) -> pd.DataFrame:
    data['year'] = data['연월일'].dt.year
    data['month'] = data['연월일'].dt.month
    data['day'] = data['연월일'].dt.day
    data['week_no'] = data['연월일'].dt.strftime("%V").astype(int)
    data['dayofweek'] = data['연월일'].dt.dayofweek
    data['weekend_yn'] = np.where(data['dayofweek'].isin(['Saturday', 'Sunday']), 1, 0)
    data['dayofyear'] = data['연월일'].dt.dayofyear

    return data


def change_dates(row):
    if row['시간'] == 24:
        row['연월일'] += pd.DateOffset(days=1)
        row['시간'] = 0
    return row


def change_pandas_date_format(df):
    """ 연월일 날짜 포맷 바꾸기"""
    df = df.apply(change_dates, axis=1)
    df['시간'] = df['시간'].apply(lambda x: str(x) if x >= 10 else '0' + str(x))
    df['연월일'] = df['연월일'].astype(str)
    df['datetime'] = df['연월일'] + ' ' + df['시간']
    df['datetime'] = pd.to_datetime(df['datetime'], format='%Y-%m-%d %H')

    df['prev_datetime'] = df['datetime'] - pd.DateOffset(years=1)

    return df


def make_autocorr_vars(df, gubun='A'):
    sub_df = df[df['구분'] == gubun]
    # 일년 전 그 시간대 공급량
    sub_df['prev_year_공급량'] = sub_df.groupby([sub_df['datetime'].dt.month, sub_df['datetime'].dt.day,
                                          sub_df['datetime'].dt.hour])['공급량'].shift()
#     # 이년 전 공급량 파생변수 생성
#     sub_df['prev_two_year_공급량'] = sub_df.groupby([sub_df['datetime'].dt.month, sub_df['datetime'].dt.day,
#                                               sub_df['datetime'].dt.hour])['공급량'].shift(2)
    #===============
    #일년 전 평균 공급량
    #===============
    sub_df['prev_year'] = sub_df['prev_datetime'].dt.year
    prev_year_mean_df = sub_df.groupby([sub_df['datetime'].dt.year])['공급량'].mean()
    prev_year_mean_dict = dict(zip(prev_year_mean_df.index, prev_year_mean_df.values.reshape(-1, )))
    sub_df['prev_year_avg_공급량'] = sub_df['prev_year'].map(prev_year_mean_dict)
    #====================
    # 일년 전 월별 평균 공급량
    #====================
    prev_month_mean_df = sub_df.groupby([sub_df['datetime'].dt.year, sub_df['datetime'].dt.month])['공급량'].mean()
    prev_month_mean_dict = dict(zip(prev_month_mean_df.index, prev_month_mean_df.values.reshape(-1, )))
    sub_df['prev_year_month'] = tuple(zip(sub_df['prev_year'].values.reshape(-1, ),
                                        sub_df['prev_datetime'].dt.month.values.reshape(-1, )))
    sub_df['prev_month_avg_공급량'] = sub_df['prev_year_month'].map(prev_month_mean_dict)
    #=======================
    # 일년 전 월-일자별 평균 공급량
    #=======================
    prev_month_day_mean_df = sub_df.groupby([sub_df['datetime'].dt.year,
                                           sub_df['datetime'].dt.month,
                                           sub_df['datetime'].dt.day])['공급량'].mean()
    prev_month_day_mean_dict = dict(zip(prev_month_day_mean_df.index, prev_month_day_mean_df.values.reshape(-1, )))

    sub_df['prev_year_month_day'] = tuple(zip(sub_df['prev_year'].values.reshape(-1, ),
                                            sub_df['prev_datetime'].dt.month.values.reshape(-1, ),
                                            sub_df['prev_datetime'].dt.day.values.reshape(-1, )))
    sub_df['prev_month_day_avg_공급량'] = sub_df['prev_year_month_day'].map(prev_month_day_mean_dict)
    #========================
    # 일년 전 월-시간 별 평균 공급량
    #========================
    prev_month_hour_mean_df = sub_df.groupby([sub_df['datetime'].dt.year,
                                            sub_df['datetime'].dt.month,
                                            sub_df['datetime'].dt.hour])['공급량'].mean()
    prev_month_hour_mean_dict = dict(zip(prev_month_hour_mean_df.index, prev_month_hour_mean_df.values.reshape(-1, )))

    sub_df['prev_year_month_hour'] = tuple(zip(sub_df['prev_year'].values.reshape(-1, ),
                                             sub_df['prev_datetime'].dt.month.values.reshape(-1, ),
                                             sub_df['prev_datetime'].dt.hour.values.reshape(-1, )))
    sub_df['prev_month_hour_avg_공급량'] = sub_df['prev_year_month_hour'].map(prev_month_hour_mean_dict)
    #==============
    # 증가율 변수 생성
    #==============
    sub_df['prev_year_ratio'] = (sub_df['prev_year_공급량']-sub_df['prev_year_avg_공급량'])/sub_df['prev_year_avg_공급량']
    sub_df['prev_month_ratio'] = (sub_df['prev_year_공급량']-sub_df['prev_month_avg_공급량'])/sub_df['prev_month_avg_공급량']
    sub_df['prev_month_day_ratio'] = (sub_df['prev_year_공급량']-sub_df['prev_month_day_avg_공급량'])/sub_df['prev_month_day_avg_공급량']
    sub_df['prev_month_hour_ratio'] = (sub_df['prev_year_공급량']-sub_df['prev_month_hour_avg_공급량'])/sub_df['prev_month_hour_avg_공급량']
    #========================
    # 일년 전 그 날의 기상 변수 FE
    #========================
    weather_cols = ['avg_temp','min_temp','max_temp','sum_rain','avg_wind','avg_humid','sum_gsr','ddmefs','avg_ts']
    for col in weather_cols:
        sub_df[f'prev_year_{col}'] = sub_df.groupby([sub_df['datetime'].dt.month, sub_df['datetime'].dt.day,
                                                  sub_df['datetime'].dt.hour])[col].shift()
        sub_df[f'prev_year_{col}'] = pd.to_numeric(sub_df[f'prev_year_{col}'], downcast="float")
    #================
    # 필요한 칼럼들만 추출
    #================
    used_cols = ['연월일', '시간', '구분', 'datetime',
                 #'year', 'month', 'day', 'week_no','dayofweek','weekend_yn','dayofyear',
                 'prev_year_공급량', 'prev_year_avg_공급량','prev_month_avg_공급량', 'prev_month_day_avg_공급량', 'prev_month_hour_avg_공급량',
                 'prev_year_ratio','prev_month_ratio','prev_month_day_ratio','prev_month_hour_ratio']
    #weather_cols = [f'prev_year_{col}' for col in weather_cols]
    #used_cols += weather_cols + ['공급량']
    used_cols += ['공급량']
    sub_df = sub_df[used_cols]
    
    return sub_df


def fe_autocorr_vars(df):
    final_df = pd.DataFrame()
    gubun_cols = ['A','B','C','D','E','G','H']
    for gubun in tqdm(gubun_cols):
        sub_df = make_autocorr_vars(df, gubun=gubun)
        final_df = pd.concat([final_df, sub_df], axis=0)
    return final_df

def change_dates_adversely(row):
    if row['datetime'].hour == 0:
        row['연월일'] -= pd.DateOffset(days=1)
        row['시간'] = 24
    return row

def change_date_format(df):
    df['연월일'] = pd.to_datetime(df['연월일'])
    df['시간'] = df['시간'].astype(int)
    df = df.apply(change_dates_adversely, axis=1)
    
    del df['datetime']
    
    return df


path = '/Users/younghun/Desktop/gitrepo/KaggleStruggle/dacon_gas'
gas_weather = merge_gas_weather(path)
dataset = make_datetime_vars(gas_weather)
dataset = change_pandas_date_format(dataset)
final_df = fe_autocorr_vars(dataset)
final_df = change_date_format(final_df)
final_df = final_df.reset_index(drop=True)

print(final_df.shape)



100%|██████████| 7/7 [00:06<00:00,  1.16it/s]


(383208, 13)


In [2]:
# 2014년부터 데이터 추출하는데, 매변 01-03월만 사용할지, 전체 사용할지 함수
def extract_dataset(final_df, split=False):
    # 2014-01-01부터 사용
    cond = final_df['연월일'].dt.year >= 2014
    final_df = final_df[cond]
    if split:
        cond0103 = (final_df['연월일'].dt.month >= 1) & (final_df['연월일'].dt.month <= 3)
        final_df = final_df[cond0103]
        return final_df
    else:
        return final_df
    
# 구분 없이 통합 예측 or 구분 유형대로 예측하기 위한 데이터 포맷 형태
def predict_simultaneously(final_df, simultaneously=False):
    if simultaneously:
        # 구분 원-핫 인코딩
        final_df[['B','C','D','E','G','H']] = pd.get_dummies(final_df['구분'], drop_first=True).values
        cols_order = final_df.columns.tolist()
        cols_order.remove('공급량')
        cols_order.insert(len(cols_order), '공급량')
        final_df = final_df[cols_order]
        return final_df
    else:
        return final_df

In [45]:
# 구분 유형별로 predict
def get_NMAE(true, pred):
    mae = np.abs(true-pred)/true
    nmae = round(np.mean(mae), 3)
    return nmae

def predict_each_gubun(final_df, model, submission_csv, simultaneously=True):
    # 통합해서 예측
    if simultaneously:
        final_df = final_df.set_index(['연월일', '시간', '구분'])
        idx_level = final_df.index.get_level_values
        #======================
        # Train, Valid로 성능 검증
        #======================
        train = final_df[(idx_level(0) < '2018-09-01')]
        train = train.fillna(method='ffill')
        valid = final_df[(idx_level(0) >=' 2018-09-01') & (idx_level(0) <= '2018-12-31')]
        # X, y 분할
        X_train, y_train = train.iloc[:, :-1].values, train['공급량']
        X_valid, y_valid = valid.iloc[:, :-1].values, valid['공급량']
        # fit
        model.fit(X_train, y_train)
        # predict
        train_pred = model.predict(X_train)
        valid_pred = model.predict(X_valid)
        # evaluate
        train_nmae = get_NMAE(y_train, train_pred)
        valid_nmae = get_NMAE(y_valid, valid_pred)
        print(f'# 통합 예측 - Train NAME:', round(train_nmae, 3))
        print(f'# 통합 예측 - Valid NMAE:', round(valid_nmae, 3))
        #======================
        # 모두 학습시키고 Test로 예측
        #======================
        train = final_df[(idx_level(0) < '2019-01-01')]
        train = train.fillna(method='ffill')
        test = final_df[(idx_level(0) >= '2019-01-01')]
        # X,y 분할
        X_train, y_train = train.iloc[:, :-1].values, train['공급량']
        X_test = test.iloc[:, :-1].fillna(method='ffill').values
        # fit
        model.fit(X_train, y_train)
        # predict
        test_pred = model.predict(X_test)
        submission_csv['공급량'] = test_pred
        return submission_csv
        
    else:
        cols = ['A', 'B', 'C', 'D', 'E', 'G', 'H']
        total_test_pred = np.array([])
        for col in cols:
            gubun_df = final_df[final_df['구분'] == col]
            gubun_df = gubun_df.set_index(['연월일','시간','구분'])

            idx_level = gubun_df.index.get_level_values
            #======================
            # Train, Valid로 성능 검증
            #======================
            train = gubun_df[(idx_level(0) < '2018-01-01')]
            train = train.fillna(method='ffill')  # 결측치 있으면 안도는 모델용
            valid = gubun_df[(idx_level(0) >= '2018-01-01') & (idx_level(0) <= '2018-03-31')]
            # X, y 분할
            X_train, y_train = train.iloc[:, :-1].values, train['공급량']
            X_valid, y_valid = valid.iloc[:, :-1].values, valid['공급량']
            # fit
            model.fit(X_train, y_train)
            # predict
            train_pred = model.predict(X_train)
            valid_pred = model.predict(X_valid)
            # evaluate
            train_nmae = get_NMAE(y_train, train_pred)
            valid_nmae = get_NMAE(y_valid, valid_pred)
            print(f'# {col} - Train NAME:', round(train_nmae, 3))
            print(f'# {col} - Valid NMAE:', round(valid_nmae, 3))
            print()
            #======================
            # 모두 학습시키고 Test로 예측
            #======================
            train = gubun_df[(idx_level(0) < '2019-01-01')]
            train = train.fillna(method='ffill')
            test = gubun_df[(idx_level(0) >= '2019-01-01')]
            # X,y 분할
            X_train, y_train = train.iloc[:, :-1].values, train['공급량']
            X_test = test.iloc[:, :-1]
            X_test = X_test.fillna(method='ffill')  # 결측치 있으면 안도는 모델용
            X_test = X_test.values
            # fit
            model.fit(X_train, y_train)
            # predict
            test_pred = model.predict(X_test)
            total_test_pred = np.append(total_test_pred, test_pred)

        submission_csv['공급량'] = total_test_pred
        return submission_csv, test

In [21]:
# model
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import VotingRegressor


# 01-03월만 추출
df = extract_dataset(final_df, split=False)
# 구분 분리안하고 동시에 예측
df = predict_simultaneously(df, simultaneously=False)

# submit
submission_csv = pd.read_csv('/Users/younghun/Desktop/gitrepo/KaggleStruggle/dacon_gas/sample_submission.csv')
# model
lgbm = LGBMRegressor(n_estimators=100, random_state=42)
xgb = XGBRegressor(n_estimators=100, random_state=42)
voting_reg = VotingRegressor(estimators=[('lgbm', lgbm),
                                        ('xgb', xgb)])

submission_csv, test = predict_each_gubun(final_df=df, model=voting_reg, submission_csv=submission_csv,
                                          simultaneously=False)

[13:34:34] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
# A - Train NAME: 0.117
# A - Valid NMAE: 0.164

[13:34:37] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:34:39] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
# B - Train NAME: 0.121
# B - Valid NMAE: 0.194

[13:34:42] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:34:45] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
# C - Train NAME: 1.622
# C - Valid NMAE: 0.149

[13:34:47] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:34:50] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
# D - Train NAME: 0.115
# D - Valid NMAE:

In [ ]:
sub_path = '/Users/younghun/Desktop/gitrepo/KaggleStruggle/dacon_gas/submission/'

# 파일명 -> 01-03월 분리여부_구분유형분리여부_모델종류_trainNMAE_validNMAE.csv
submission_csv.to_csv(sub_path+'0103False_SimulFalse_xgb+lgbm_cols-autocorr_ratio.csv', index=False)

## 현재 상태
- 현재 가장 좋은 성능: 0.12679
- 사용한 변수:
    - 1년 전 자기상관 변수들
    - 1년 전 자기상관 변수들 간의 변화율
- 사용한 모델:
    - XGBoost + LightGBM 하이브리드 Voting
<br><br>
- ~~Stacking 모델 시도~~
- Prophet 시도
- 구분 유형별로 모델 종류 달리해서 시도

### Stacking
- Single 모델
    - XGBoost
    - LightGBM
    - Random Forest
- 최종 메타 모델: HybridVoting(XGBoost+LightGBM)
- Test 성능 0.138로 이전보다 오버피팅 발생..

In [49]:
# 1. XGBoost로 학습 데이터에 대한 예측값, 테스트 데이터에 대한 예측값 반환

def predict_single_model(final_df, idx_level, model):
    #=============================
    # Train, Validation 나누기
    #=============================
    train_v = final_df[(idx_level(0) < '2018-09-01')].fillna(method='ffill')
    valid_v = final_df[(idx_level(0) >= '2018-09-01') & (idx_level(0) <= '2018-12-31')].fillna(method='ffill')
    # X, y 분할
    X_train_v, y_train_v = train_v.iloc[:, :-1].values, train_v['공급량']
    X_valid_v, y_valid_v = valid_v.iloc[:, :-1].values, valid_v['공급량']
    # fit
    model.fit(X_train_v, y_train_v)
    # predict
    train_pred_v = model.predict(X_train_v)
    valid_pred_v = model.predict(X_valid_v)
    
    #=============================
    # Train, Test로 나누어 학습 및 예측
    #=============================
    train = final_df[(idx_level(0) < '2019-01-01')].fillna(method='ffill')
    test = final_df[(idx_level(0) >= '2019-01-01')].fillna(method='ffill')
    print('# Train shape:', train.shape, '# Test shape:', test.shape)
    # X, y 분할
    X_train, y_train = train.iloc[:, :-1].values, train['공급량']
    X_test = test.iloc[:, :-1].values
    # fit
    model.fit(X_train, y_train)
    # predict
    train_pred = model.predict(X_train)
    test_pred = model.predict(X_test)
    print('### Model:', model)
    print('# Train pred shape:', train_pred.shape, '# Test pred shape:', test_pred.shape)
    print()
    return train_pred_v, valid_pred_v, train_pred, test_pred


def predict_stacking(final_df, meta_model, single_models, submission_csv):
    # Multiindex로 변경
    final_df = final_df.set_index(['연월일', '시간', '구분'])
    idx_level = final_df.index.get_level_values
    # 메타모델 성능 체크하기 위해 y값 따로 떼놓기
    y_train_v = final_df[(idx_level(0) < '2018-09-01')]['공급량'].values
    y_valid_v = final_df[(idx_level(0) >= '2018-09-01') & (idx_level(0) <= '2018-12-31')]['공급량'].values
    # 메타모델 최종 성능 학습 및 테스트 하기 위해 y값 따로 떼놓기
    y_train = final_df[(idx_level(0) < '2019-01-01')]['공급량'].values
    y_test = final_df[(idx_level(0) >= '2019-01-01')]['공급량'].values  # 최종 예측할 값들이므로 NaN값들임!
    
    #==================================
    # 메타모델의 학습데이터로 사용될 행렬들 초기화
    #==================================
    # 메타모델 성능 검증용 초기화 행렬
    X_train_v = np.zeros((y_train_v.shape[0], len(single_models)))
    X_valid_v = np.zeros((y_valid_v.shape[0], len(single_models)))
    # 메타모델 최종 학습 및 테스트 초기화 행렬
    X_train = np.zeros((y_train.shape[0], len(single_models)))
    X_test = np.zeros((y_test.shape[0], len(single_models)))
    
    #=======================
    # single model 하나씩 수행
    #=======================
    for i, single_model in enumerate(single_models):
        train_pred_v, valid_pred_v, train_pred, test_pred = predict_single_model(final_df, idx_level, single_model)
        # numpy array concatenate
        X_train_v[:, i] = train_pred_v
        X_valid_v[:, i] = valid_pred_v
        X_train[:, i] = train_pred
        X_test[:, i] = test_pred
    
    #==============================
    # 메타모델로 최종 성능 검증 및 최종 예측
    #==============================
    # 성능 검증
    meta_model.fit(X_train_v, y_train_v)
    TRAIN_V_PRED = meta_model.predict(X_train_v)
    VALID_V_PRED = meta_model.predict(X_valid_v)
    TRAIN_V_NMAE = get_NMAE(y_train_v, TRAIN_V_PRED)
    VALID_V_NMAE = get_NMAE(y_valid_v, VALID_V_PRED)
    print('🔥최종 메타 모델 성능 검증- Train NMAE:', TRAIN_V_NMAE)
    print('🔥최종 메타 모델 성능 검증- Valid NMAE:', VALID_V_NMAE)
    
    # 최종 예측
    meta_model.fit(X_train, y_train)
    FINAL_TEST_PRED = meta_model.predict(X_test)
    
    # 제출 csv 파일에 담기
    submission_csv['공급량'] = FINAL_TEST_PRED
    
#     print('메타모델에 사용될 데이터들')
#     print('X_train_v:', X_train_v.shape, 'X_valid_v:', X_valid_v.shape)
#     print('y_train_v:', y_train_v.shape, 'y_valid_v:', y_valid_v.shape)
#     print()
#     print('X_train:', X_train.shape, 'X_test:', X_test.shape)
#     print('y_train:', y_train.shape, 'y_test:', y_test.shape)
    
    return submission_csv
    

lgbm = LGBMRegressor(n_estimators=100, random_state=42)
xgb = XGBRegressor(n_estimators=100, random_state=42)
rf = RandomForestRegressor(random_state=42)

META_MODEL = VotingRegressor([('lgbm', lgbm), ('xgb', xgb)])
single_models = [lgbm, xgb, rf]

submission_csv = pd.read_csv('/Users/younghun/Desktop/gitrepo/KaggleStruggle/dacon_gas/sample_submission.csv')

submission_csv = predict_stacking(final_df=df, meta_model=META_MODEL, 
                                  single_models=single_models, submission_csv=submission_csv)

# Train shape: (306768, 10) # Test shape: (15120, 10)
### Model: LGBMRegressor(random_state=42)
# Train pred shape: (306768,) # Test pred shape: (15120,)

[14:24:09] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
# Train shape: (306768, 10) # Test shape: (15120, 10)
[14:24:29] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
### Model: XGBRegressor(random_state=42)
# Train pred shape: (306768,) # Test pred shape: (15120,)

# Train shape: (306768, 10) # Test shape: (15120, 10)
### Model: RandomForestRegressor(random_state=42)
# Train pred shape: (306768,) # Test pred shape: (15120,)

[14:33:11] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
🔥최종 메타 모델 성능 검증- Train NMAE: 0.1
🔥최종 메타 모델 성능 검증- Valid NMAE: 0.191
[14:33:24] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederr

In [51]:
sub_path = '/Users/younghun/Desktop/gitrepo/KaggleStruggle/dacon_gas/submission/'

# 파일명 -> 01-03월 분리여부_구분유형분리여부_모델종류_trainNMAE_validNMAE.csv
submission_csv.to_csv(sub_path+'0103False_SimulFalse_Stacking(xgb+lgbm+rf_meta:voting)_cols-autocorr_ratio.csv', index=False)


In [52]:
# 수치값들 스케일링 문법 추가하고 스태킹 다시 한 번 시도!?